In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
df = pd.read_csv("charging_sessions.csv")

Erstmal Infos sammeln

In [4]:
df.head()

,Unnamed: 0,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
3,3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
4,4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        66450 non-null  int64  
 1   id                66450 non-null  object 
 2   connectionTime    66450 non-null  object 
 3   disconnectTime    66450 non-null  object 
 4   doneChargingTime  62362 non-null  object 
 5   kWhDelivered      66450 non-null  float64
 6   sessionID         66450 non-null  object 
 7   siteID            66450 non-null  int64  
 8   spaceID           66450 non-null  object 
 9   stationID         66450 non-null  object 
 10  timezone          66450 non-null  object 
 11  userID            49187 non-null  float64
 12  userInputs        49187 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 6.6+ MB


In [6]:
fehlende_werte = df.isnull().sum()
fehlende_werte

Unnamed: 0              0
id                      0
connectionTime          0
disconnectTime          0
doneChargingTime     4088
kWhDelivered            0
sessionID               0
siteID                  0
spaceID                 0
stationID               0
timezone                0
userID              17263
userInputs          17263
dtype: int64

Checken ob alle ids und die Spalte unnamed unique Einträge hat

In [7]:
duplicates = df["Unnamed: 0"].unique()
len(duplicates)

15292

Die Spalte "Unnamed: 0" geht von 0 bis 15292 und wiederholt sich dann. Also löschen wir die komplett

In [8]:
df.drop(columns="Unnamed: 0", inplace=True)
df.head()

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil..."


Die fehlenden Werte in "doneChargingTime" sind vermutlich entstanden, wenn jemand weitergefahren ist, obwohl ein Auto noch nicht zu 100% aufgeladen war.

Die Zeiten werden auf datetime umgeformt, damit wir später einfacher damit rechnen können

In [9]:
df["connectionTime"] = pd.to_datetime(df["connectionTime"])
df["disconnectTime"] = pd.to_datetime(df["disconnectTime"])
df["doneChargingTime"] = pd.to_datetime(df["doneChargingTime"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   id                66450 non-null  object             
 1   connectionTime    66450 non-null  datetime64[ns, UTC]
 2   disconnectTime    66450 non-null  datetime64[ns, UTC]
 3   doneChargingTime  62362 non-null  datetime64[ns, UTC]
 4   kWhDelivered      66450 non-null  float64            
 5   sessionID         66450 non-null  object             
 6   siteID            66450 non-null  int64              
 7   spaceID           66450 non-null  object             
 8   stationID         66450 non-null  object             
 9   timezone          66450 non-null  object             
 10  userID            49187 non-null  float64            
 11  userInputs        49187 non-null  object             
dtypes: datetime64[ns, UTC](3), float64(2), int64(1), object(6)
m

Checking for invalid data

Possible invalid scenarios are:

    1. connectionTime is after disconnectTime
    2. connectionTime is after doneChargingTime
    3. doneChargingTime is after disconnectTime
    4. The same stationID has two overlapping sessions

In [10]:
disAfterCon = df["connectionTime"] > df["disconnectTime"]
disAfterCon.value_counts()

False    66450
Name: count, dtype: int64

In [11]:
disAfterDone = df["connectionTime"] > df["doneChargingTime"]
disAfterDone.value_counts()

False    66423
True        27
Name: count, dtype: int64

^ TODO: Zeilen untersuchen und evtl löschen

In [12]:
doneAfterDis = df["doneChargingTime"] > df["disconnectTime"]
doneAfterDis.value_counts()

False    61758
True      4692
Name: count, dtype: int64

^ disconnected before charging was done. Not really invalid

now checking for overlapping sessions on the same station

In [13]:
df.sort_values(by=["stationID", "connectionTime"],inplace=True)

In [14]:
df.head()

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
51769,5c366251f9af8b4639a8e16e,2018-09-13 15:24:32+00:00,2018-09-13 22:58:36+00:00,NaT,13.056,1_1_178_817_2018-09-13 15:24:32.185314,1,AG-1F09,1-1-178-817,America/Los_Angeles,NaN,NaN
51812,5c366258f9af8b4639a8e199,2018-09-14 14:26:27+00:00,2018-09-14 23:41:53+00:00,NaT,9.309,1_1_178_817_2018-09-14 14:26:26.716890,1,AG-1F09,1-1-178-817,America/Los_Angeles,NaN,NaN
51853,5c36626bf9af8b4639a8e1c2,2018-09-17 14:08:14+00:00,2018-09-17 23:54:50+00:00,NaT,35.658,1_1_178_817_2018-09-17 14:08:14.125501,1,AG-1F09,1-1-178-817,America/Los_Angeles,378.0,"[{'WhPerMile': 250, 'kWhRequested': 50.0, 'mil..."
51906,5c366274f9af8b4639a8e1f7,2018-09-18 13:43:48+00:00,2018-09-18 22:32:43+00:00,NaT,4.337,1_1_178_817_2018-09-18 13:43:47.796335,1,AG-1F09,1-1-178-817,America/Los_Angeles,405.0,"[{'WhPerMile': 500, 'kWhRequested': 10.0, 'mil..."
51950,5c366274f9af8b4639a8e223,2018-09-18 23:05:54+00:00,2018-09-19 03:06:39+00:00,NaT,9.974,1_1_178_817_2018-09-18 23:05:53.946381,1,AG-1F09,1-1-178-817,America/Los_Angeles,467.0,"[{'WhPerMile': 425, 'kWhRequested': 12.75, 'mi..."


In [15]:
df["overlap"] = ((df["connectionTime"].shift(-1) < df["disconnectTime"]) & (df["stationID"]==df["stationID"].shift(-1))).fillna(False)

In [16]:
df["overlap"].value_counts()

overlap
False    65036
True      1414
Name: count, dtype: int64

Seems like 1414 entries are overlapping

New feature ParkDuration to check for outliers

In [17]:
df["parkDuration"] = df["disconnectTime"] - df["connectionTime"]
df.head()

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs,overlap,parkDuration
51769,5c366251f9af8b4639a8e16e,2018-09-13 15:24:32+00:00,2018-09-13 22:58:36+00:00,NaT,13.056,1_1_178_817_2018-09-13 15:24:32.185314,1,AG-1F09,1-1-178-817,America/Los_Angeles,NaN,NaN,False,0 days 07:34:04
51812,5c366258f9af8b4639a8e199,2018-09-14 14:26:27+00:00,2018-09-14 23:41:53+00:00,NaT,9.309,1_1_178_817_2018-09-14 14:26:26.716890,1,AG-1F09,1-1-178-817,America/Los_Angeles,NaN,NaN,False,0 days 09:15:26
51853,5c36626bf9af8b4639a8e1c2,2018-09-17 14:08:14+00:00,2018-09-17 23:54:50+00:00,NaT,35.658,1_1_178_817_2018-09-17 14:08:14.125501,1,AG-1F09,1-1-178-817,America/Los_Angeles,378.0,"[{'WhPerMile': 250, 'kWhRequested': 50.0, 'mil...",False,0 days 09:46:36
51906,5c366274f9af8b4639a8e1f7,2018-09-18 13:43:48+00:00,2018-09-18 22:32:43+00:00,NaT,4.337,1_1_178_817_2018-09-18 13:43:47.796335,1,AG-1F09,1-1-178-817,America/Los_Angeles,405.0,"[{'WhPerMile': 500, 'kWhRequested': 10.0, 'mil...",False,0 days 08:48:55
51950,5c366274f9af8b4639a8e223,2018-09-18 23:05:54+00:00,2018-09-19 03:06:39+00:00,NaT,9.974,1_1_178_817_2018-09-18 23:05:53.946381,1,AG-1F09,1-1-178-817,America/Los_Angeles,467.0,"[{'WhPerMile': 425, 'kWhRequested': 12.75, 'mi...",False,0 days 04:00:45


In [18]:
df["parkDuration"].describe()

count                        66450
mean     0 days 06:17:34.578209179
std      0 days 04:51:41.391314114
min                0 days 00:02:04
25%         0 days 02:51:57.750000
50%                0 days 06:11:44
75%         0 days 09:13:31.500000
max               10 days 05:16:09
Name: parkDuration, dtype: object

Check for Outliers

In [19]:
from datetime import timedelta

print('park duration < 5min: ',len(df[df["parkDuration"]<timedelta(minutes=5)]))
print('park duration > 3h: ',len(df[df["parkDuration"]>timedelta(hours=3)]))
print('park duration > 6h: ',len(df[df["parkDuration"]>timedelta(hours=6)]))
print('park duration > 9h: ',len(df[df["parkDuration"]>timedelta(hours=9)]))
print('park duration > 12h: ',len(df[df["parkDuration"]>timedelta(hours=12)]))
print('park duration > 1d: ',len(df[df["parkDuration"]>timedelta(days=1)]))
print('park duration > 2d: ',len(df[df["parkDuration"]>timedelta(days=2)]))

park duration < 5min:  16
park duration > 3h:  49077
park duration > 6h:  33938
park duration > 9h:  18873
park duration > 12h:  2180
park duration > 1d:  279
park duration > 2d:  92


seems like 16 entries were shorter than 5 mins and 279 were longer than 1 day, of which 92 were longer than 2 days

In [25]:
print('park duration > 5d: ',len(df[df["parkDuration"]>timedelta(days=5)]))

park duration > 5d:  7


7 even stayed longer than 5 days ---- TODO what to do with those entries? When do we cut off?